In [95]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from itertools import combinations
import lightning.pytorch as pl
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from cca_zoo.linear import TCCA
from cca_zoo.nonparametric import KCCA
from cca_zoo.deep import (
    architectures,
    DCCA
)

In [96]:
data = load_breast_cancer()
X = data.data
df = pd.DataFrame(X, columns=data.feature_names)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [97]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

threshold = 1.5

df_processed = df[~((df < (Q1 - threshold * IQR)) | (df > (Q3 + threshold * IQR))).any(axis=1)]
ys = data.target[df_processed.index]

df_processed.info()
print(np.shape(ys))

<class 'pandas.core.frame.DataFrame'>
Index: 398 entries, 6 to 566
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              398 non-null    float64
 1   mean texture             398 non-null    float64
 2   mean perimeter           398 non-null    float64
 3   mean area                398 non-null    float64
 4   mean smoothness          398 non-null    float64
 5   mean compactness         398 non-null    float64
 6   mean concavity           398 non-null    float64
 7   mean concave points      398 non-null    float64
 8   mean symmetry            398 non-null    float64
 9   mean fractal dimension   398 non-null    float64
 10  radius error             398 non-null    float64
 11  texture error            398 non-null    float64
 12  perimeter error          398 non-null    float64
 13  area error               398 non-null    float64
 14  smoothness error         398 no

In [98]:
X_train = df_processed[:300]
y_train = ys[:300]
X_test = df_processed[300:]
y_test = ys[300:]

In [99]:
scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)

In [100]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9795918367346939


In [101]:
data_train_dict = {
    'radius': X_train[['mean radius', 'radius error', 'worst radius']].values,
    'texture': X_train[['mean texture', 'texture error', 'worst texture']].values,
    'perimeter': X_train[['mean perimeter', 'perimeter error', 'worst perimeter']].values,
    'area': X_train[['mean area', 'area error', 'worst area']].values,
    'smoothness': X_train[['mean smoothness', 'smoothness error', 'worst smoothness']].values,
    'compactness': X_train[['mean compactness', 'compactness error', 'worst compactness']].values,
    'concavity': X_train[['mean concavity', 'concavity error', 'worst concavity']].values,
    'concave_points': X_train[['mean concave points', 'concave points error', 'worst concave points']].values,
    'symmetry': X_train[['mean symmetry', 'symmetry error', 'worst symmetry']].values,
    'fractal_dimension': X_train[['mean fractal dimension', 'fractal dimension error', 'worst fractal dimension']].values
}

data_test_dict = {
    'radius': X_test[['mean radius', 'radius error', 'worst radius']].values,
    'texture': X_test[['mean texture', 'texture error', 'worst texture']].values,
    'perimeter': X_test[['mean perimeter', 'perimeter error', 'worst perimeter']].values,
    'area': X_test[['mean area', 'area error', 'worst area']].values,
    'smoothness': X_test[['mean smoothness', 'smoothness error', 'worst smoothness']].values,
    'compactness': X_test[['mean compactness', 'compactness error', 'worst compactness']].values,
    'concavity': X_test[['mean concavity', 'concavity error', 'worst concavity']].values,
    'concave_points': X_test[['mean concave points', 'concave points error', 'worst concave points']].values,
    'symmetry': X_test[['mean symmetry', 'symmetry error', 'worst symmetry']].values,
    'fractal_dimension': X_test[['mean fractal dimension', 'fractal dimension error', 'worst fractal dimension']].values
}

### Kernel CCA

In [102]:
data_train_tuple = tuple(data_train_dict.values())
data_test_tuple = tuple(data_test_dict.values())

KCCA_model = KCCA()
X_kcca_train = KCCA_model.fit_transform(data_train_tuple)
X_kcca_test = KCCA_model.transform(data_test_tuple)

X_kcca_train_log = np.squeeze(np.transpose(X_kcca_train, axes=(1, 0, 2)), axis=-1)
X_kcca_test_log = np.squeeze(np.transpose(X_kcca_test, axes=(1, 0, 2)), axis=-1)

log_reg = LogisticRegression()
log_reg.fit(X_kcca_train_log, y_train)
y_kcca_pred = log_reg.predict(X_kcca_test_log)

accuracy = accuracy_score(y_test, y_kcca_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8163265306122449


In [103]:
all_attributes = list(data_train_dict.keys())
all_subsets = combinations(all_attributes, 2)

data_train_subsets = {}
data_test_subsets = {}

for subset in all_subsets:
    subset_name = "_".join(subset)
    subset_train_values = [data_train_dict[attr] for attr in subset]
    subset_test_values = [data_test_dict[attr] for attr in subset]
    data_train_subsets[subset_name] = subset_train_values
    data_test_subsets[subset_name] = subset_test_values

best_accuracy = 0
best_subset = None

for subset_name, subset_values in data_train_subsets.items():
    data_train_tuple = tuple(subset_values)
    data_test_tuple = tuple(data_test_subsets[subset_name])

    KCCA_model = KCCA()
    X_train_transformed = KCCA_model.fit_transform(data_train_tuple)
    X_test_transformed = KCCA_model.transform(data_test_tuple)

    X_train_log = np.squeeze(np.transpose(X_train_transformed, axes=(1, 0, 2)), axis=-1)
    X_test_log = np.squeeze(np.transpose(X_test_transformed, axes=(1, 0, 2)), axis=-1)

    log_reg = LogisticRegression()
    log_reg.fit(X_train_log, y_train)

    y_pred = log_reg.predict(X_test_log)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{subset_name} - {accuracy}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_subset = subset_name

print("----\----")
print("Tập có accuracy cao nhất:", best_subset)
print("Accuracy:", best_accuracy)

<>:39: SyntaxWarning: invalid escape sequence '\-'
<>:39: SyntaxWarning: invalid escape sequence '\-'
C:\Users\nguyn\AppData\Local\Temp\ipykernel_19436\2540473077.py:39: SyntaxWarning: invalid escape sequence '\-'
  print("----\----")


radius_texture - 0.8061224489795918
radius_perimeter - 0.9081632653061225
radius_area - 0.9183673469387755
radius_smoothness - 0.8061224489795918
radius_compactness - 0.7857142857142857
radius_concavity - 0.8775510204081632
radius_concave_points - 0.8367346938775511
radius_symmetry - 0.7755102040816326
radius_fractal_dimension - 0.7755102040816326
texture_perimeter - 0.8163265306122449
texture_area - 0.8469387755102041
texture_smoothness - 0.826530612244898
texture_compactness - 0.7959183673469388
texture_concavity - 0.826530612244898
texture_concave_points - 0.7959183673469388
texture_symmetry - 0.7857142857142857
texture_fractal_dimension - 0.7959183673469388
perimeter_area - 0.9183673469387755
perimeter_smoothness - 0.8061224489795918
perimeter_compactness - 0.8061224489795918
perimeter_concavity - 0.8673469387755102
perimeter_concave_points - 0.8469387755102041
perimeter_symmetry - 0.7959183673469388
perimeter_fractal_dimension - 0.8163265306122449
area_smoothness - 0.8265306122448

### Deep CCA

In [104]:
class DictDataset(Dataset):
    def __init__(self, data_dict):
        self.data_dict = data_dict
        self.keys = list(data_dict.keys())

    def __len__(self):
        return len(self.data_dict[self.keys[0]])

    def __getitem__(self, index):
        sample = {key: self.data_dict[key][index] for key in self.keys}
        return sample

all_attributes = list(data_train_dict.keys())
all_subsets = combinations(all_attributes, 2)

data_train_subsets = {}
data_test_subsets = {}

for subset in all_subsets:
    subset_name = "_".join(subset)
    subset_train_values = [data_train_dict[attr] for attr in subset]
    subset_test_values = [data_test_dict[attr] for attr in subset]
    data_train_subsets[subset_name] = subset_train_values
    data_test_subsets[subset_name] = subset_test_values

best_accuracy = 0
best_subset = None

res = {}

for subset_name, subset_values in data_train_subsets.items():
    print(subset_name)
    
    views_tensors = [value.astype(np.float32) for value in subset_values]
    test_views = [value.astype(np.float32) for value in data_test_subsets[subset_name]]
    label_tensor = y_train
    index_tensor = np.arange(len(y_train))
        
    custom_dataset = {
        "views": views_tensors,
        "label": label_tensor,
        "index": index_tensor
    }
    train_dataset = DictDataset(custom_dataset)

    trainloader = DataLoader(train_dataset, batch_size=32, shuffle=False)

    latent_dims = 10
    max_epoch = 30

    encoder_1 = architectures.Encoder(latent_dimensions=latent_dims, feature_size=3)
    encoder_2 = architectures.Encoder(latent_dimensions=latent_dims, feature_size=3)

    dcca = DCCA(latent_dimensions=latent_dims, encoders=[encoder_1, encoder_2])
    trainer = pl.Trainer(
        max_epochs=max_epoch,
        enable_checkpointing=False,
        enable_model_summary=False,
        enable_progress_bar=False,
    )
    trainer.fit(dcca, trainloader)

    X_train_dcca = dcca.transform(trainloader)

    test_dataset = DictDataset({
        "views": test_views
    })

    testloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    X_test_dcca = dcca.transform(testloader)

    merged_list = []

    for i in range(len(X_train_dcca[0])):
        new_sublist = []
        for sublist_group in X_train_dcca:
            new_sublist.extend(sublist_group[i])
        merged_list.append(new_sublist)

    X_train_dcca = merged_list

    merged_list = []

    for i in range(len(X_test_dcca[0])):
        new_sublist = []
        for sublist_group in X_test_dcca:
            new_sublist.extend(sublist_group[i])
        merged_list.append(new_sublist)

    X_test_dcca = merged_list

    log_reg = LogisticRegression()
    log_reg.fit(X_train_dcca, y_train)

    y_dcca_pred = log_reg.predict(X_test_dcca)
    accuracy = accuracy_score(y_test, y_dcca_pred)
    res[subset_name] = accuracy

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_subset = subset_name

print(res)

print("----\----")
print("Tập có accuracy cao nhất:", best_subset)
print("Accuracy:", best_accuracy)

<>:105: SyntaxWarning: invalid escape sequence '\-'
<>:105: SyntaxWarning: invalid escape sequence '\-'
C:\Users\nguyn\AppData\Local\Temp\ipykernel_19436\2937504000.py:105: SyntaxWarning: invalid escape sequence '\-'
  print("----\----")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


radius_texture


c:\Users\nguyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
c:\Users\nguyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\nguyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=30` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using

radius_perimeter


`Trainer.fit` stopped: `max_epochs=30` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\nguyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
c:\Users\nguyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\nguyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if 

...

concave_points_fractal_dimension


c:\Users\nguyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\nguyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=30` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\nguyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:74: You defined a `validation_step` but 

symmetry_fractal_dimension


`Trainer.fit` stopped: `max_epochs=30` reached.


{'radius_texture': 0.9285714285714286, 'radius_perimeter': 0.9183673469387755, 'radius_area': 0.9183673469387755, 'radius_smoothness': 0.9387755102040817, 'radius_compactness': 0.9387755102040817, 'radius_concavity': 0.9591836734693877, 'radius_concave_points': 0.9489795918367347, 'radius_symmetry': 0.9387755102040817, 'radius_fractal_dimension': 0.9489795918367347, 'texture_perimeter': 0.9489795918367347, 'texture_area': 0.9489795918367347, 'texture_smoothness': 0.826530612244898, 'texture_compactness': 0.8979591836734694, 'texture_concavity': 0.9591836734693877, 'texture_concave_points': 0.9795918367346939, 'texture_symmetry': 0.826530612244898, 'texture_fractal_dimension': 0.8571428571428571, 'perimeter_area': 0.9285714285714286, 'perimeter_smoothness': 0.9387755102040817, 'perimeter_compactness': 0.9285714285714286, 'perimeter_concavity': 0.9591836734693877, 'perimeter_concave_points': 0.9489795918367347, 'perimeter_symmetry': 0.9387755102040817, 'perimeter_fractal_dimension': 0.95

### Tensor CCA

In [105]:
def generate_subsets(attributes):
    subsets = []
    for r in range(1, len(attributes) + 1):
        subsets.extend(combinations(attributes, r))
    return subsets

all_attributes = list(data_train_dict.keys())
all_subsets = generate_subsets(all_attributes)

data_train_subsets = {}
data_test_subsets = {}

for subset in all_subsets:
    subset_name = "_".join(subset)
    subset_train_values = [data_train_dict[attr] for attr in subset]
    subset_test_values = [data_test_dict[attr] for attr in subset]
    data_train_subsets[subset_name] = subset_train_values
    data_test_subsets[subset_name] = subset_test_values

In [106]:
best_accuracy = 0
best_subset = None

res = {}

for subset_name, subset_values in data_train_subsets.items():
    data_train_tuple = tuple(subset_values)
    data_test_tuple = tuple(data_test_subsets[subset_name])

    TCCA_model = TCCA()
    X_train_transformed = TCCA_model.fit_transform(data_train_tuple)
    X_test_transformed = TCCA_model.transform(data_test_tuple)

    X_train_log = np.squeeze(np.transpose(X_train_transformed, axes=(1, 0, 2)), axis=-1)
    X_test_log = np.squeeze(np.transpose(X_test_transformed, axes=(1, 0, 2)), axis=-1)

    log_reg = LogisticRegression()
    log_reg.fit(X_train_log, y_train)

    y_pred = log_reg.predict(X_test_log)
    accuracy = accuracy_score(y_test, y_pred)
    res[subset_name] = accuracy

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_subset = subset_name

print(res)
    
print("----\----")
print("Tập có accuracy cao nhất:", best_subset)
print("Accuracy:", best_accuracy)

<>:30: SyntaxWarning: invalid escape sequence '\-'
<>:30: SyntaxWarning: invalid escape sequence '\-'
C:\Users\nguyn\AppData\Local\Temp\ipykernel_19436\821506387.py:30: SyntaxWarning: invalid escape sequence '\-'
  print("----\----")
c:\Users\nguyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\nguyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increas

{'radius': 0.826530612244898, 'texture': 0.826530612244898, 'perimeter': 0.826530612244898, 'area': 0.826530612244898, 'smoothness': 0.826530612244898, 'compactness': 0.826530612244898, 'concavity': 0.826530612244898, 'concave_points': 0.826530612244898, 'symmetry': 0.826530612244898, 'fractal_dimension': 0.826530612244898, 'radius_texture': 0.8163265306122449, 'radius_perimeter': 0.9081632653061225, 'radius_area': 0.9183673469387755, 'radius_smoothness': 0.8061224489795918, 'radius_compactness': 0.7857142857142857, 'radius_concavity': 0.8775510204081632, 'radius_concave_points': 0.8367346938775511, 'radius_symmetry': 0.7653061224489796, 'radius_fractal_dimension': 0.7653061224489796, 'texture_perimeter': 0.8163265306122449, 'texture_area': 0.8469387755102041, 'texture_smoothness': 0.826530612244898, 'texture_compactness': 0.7959183673469388, 'texture_concavity': 0.826530612244898, 'texture_concave_points': 0.7959183673469388, 'texture_symmetry': 0.7857142857142857, 'texture_fractal_di

c:\Users\nguyn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
